# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [2]:
from sklearn import cross_validation, datasets, grid_search, linear_model, metrics

import numpy as np
import pandas as pd

/home/igor/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/igor/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Генерация датасета

In [3]:
iris = datasets.load_iris()

In [4]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [5]:
classifier = linear_model.SGDClassifier(random_state = 0)

### Генерация сетки

In [6]:
classifier.get_params().keys()

['warm_start',
 'loss',
 'n_jobs',
 'eta0',
 'verbose',
 'shuffle',
 'fit_intercept',
 'epsilon',
 'average',
 'max_iter',
 'penalty',
 'power_t',
 'random_state',
 'tol',
 'l1_ratio',
 'n_iter',
 'alpha',
 'learning_rate',
 'class_weight']

In [7]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'n_iter' : range(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [8]:
cv = cross_validation.StratifiedShuffleSplit(train_labels, n_iter = 10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [12]:
grid_cv = grid_search.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [30]:
import warnings
?warnings.simplefilter
?grid_search.GridSearchCV

Init signature: grid_search.GridSearchCV(self, estimator, param_grid, scoring=None, fit_params=None, n_jobs=1, iid=True, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score='raise')
Docstring:     
Exhaustive search over specified parameter values for an estimator.

.. deprecated:: 0.18
    This module will be removed in 0.20.
    Use :class:`sklearn.model_selection.GridSearchCV` instead.

Important members are fit, predict.

GridSearchCV implements a "fit" and a "score" method.
It also implements "predict", "predict_proba", "decision_function",
"transform" and "inverse_transform" if they are implemented in the
estimator used.

The parameters of the estimator used to apply these methods are optimized
by cross-validated grid-search over a parameter grid.

Read more in the :ref:`User Guide <grid_search>`.

Parameters
----------
estimator : estimator object.
    A object of that type is instantiated for each grid point.
    This is assumed to implement the scikit-learn es

In [23]:
%%time
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    grid_cv.fit(train_data, train_labels)

CPU times: user 7.81 s, sys: 16.9 ms, total: 7.83 s
Wall time: 7.83 s


In [24]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.000325, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=9,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=0, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [25]:
print grid_cv.best_score_
print grid_cv.best_params_

0.895238095238
{'penalty': 'l1', 'alpha': 0.000325, 'n_iter': 9, 'loss': 'hinge'}


In [26]:
grid_cv.grid_scores_[:10]

[mean: 0.75714, std: 0.13544, params: {'penalty': 'l1', 'alpha': 0.0001, 'n_iter': 5, 'loss': 'hinge'},
 mean: 0.66667, std: 0.15936, params: {'penalty': 'l2', 'alpha': 0.0001, 'n_iter': 5, 'loss': 'hinge'},
 mean: 0.61429, std: 0.14357, params: {'penalty': 'l1', 'alpha': 0.0001, 'n_iter': 6, 'loss': 'hinge'},
 mean: 0.68571, std: 0.15386, params: {'penalty': 'l2', 'alpha': 0.0001, 'n_iter': 6, 'loss': 'hinge'},
 mean: 0.79524, std: 0.16503, params: {'penalty': 'l1', 'alpha': 0.0001, 'n_iter': 7, 'loss': 'hinge'},
 mean: 0.70952, std: 0.18982, params: {'penalty': 'l2', 'alpha': 0.0001, 'n_iter': 7, 'loss': 'hinge'},
 mean: 0.69524, std: 0.19772, params: {'penalty': 'l1', 'alpha': 0.0001, 'n_iter': 8, 'loss': 'hinge'},
 mean: 0.63810, std: 0.17587, params: {'penalty': 'l2', 'alpha': 0.0001, 'n_iter': 8, 'loss': 'hinge'},
 mean: 0.84286, std: 0.12608, params: {'penalty': 'l1', 'alpha': 0.0001, 'n_iter': 9, 'loss': 'hinge'},
 mean: 0.78095, std: 0.10690, params: {'penalty': 'l2', 'alpha':

#### Randomized grid search

In [27]:
randomized_grid_cv = grid_search.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0)

In [28]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

CPU times: user 759 ms, sys: 3.39 ms, total: 763 ms
Wall time: 829 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(labels=[1 2 ... 2 0], n_iter=10, test_size=0.2, random_state=0),
          error_score='raise',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=None, verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=20, n_jobs=1,
          param_distributions={'penalty': ['l1', 'l2'], 'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'alpha': array([0.0001 , 0.00032, 0.00055, 0.00078, 0.001  ]), 'n_iter': [5, 6, 7, 8, 9]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          scoring='accuracy', verbose=0)

In [29]:
print randomized_grid_cv.best_score_
print randomized_grid_cv.best_params_

0.814285714286
{'penalty': 'l1', 'n_iter': 9, 'alpha': 0.00055, 'loss': 'log'}
